In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from time import time
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()


df = pd.read_csv("../data_process/merged_AAPL_30m_191001_191231.csv")
df.drop(columns=['<VOL>'], inplace=True)
# df = df.iloc[: , 1:]
df.sort_values(by='<TIME>', ascending=True, inplace=True)
df.set_index("<TIME>", inplace=True)

print("---dataframe head---")
print(df.head())

print("--scaling data---")
data = sc.fit_transform(df)

train_ind = int(0.8 * len(df))
# val_ind = train_ind + int(0.2*len(df))

train = data[:train_ind]
# val = data[train_ind:val_ind]
# test = data[val_ind:]
test = data[train_ind:]

print("--shapes--")
print("train, test", train.shape, test.shape)

---dataframe head---
                     <OPEN>   <HIGH>     <LOW>   <CLOSE>  tw_count  \
<TIME>                                                               
2019-10-01 10:00:00  225.13  228.220  224.7000  228.0700         9   
2019-10-01 10:30:00  228.05  228.080  225.8389  226.7554         3   
2019-10-01 11:00:00  226.75  227.135  226.0600  226.1900         9   
2019-10-01 11:30:00  226.18  227.640  225.9600  227.6400        13   
2019-10-01 12:00:00  227.64  227.660  226.0700  226.2700        11   

                     tw_mean_chars  tw_n_pos  tw_ratio_pos  tw_n_neg  \
<TIME>                                                                 
2019-10-01 10:00:00      98.222222         1      0.111111         0   
2019-10-01 10:30:00      61.000000         1      0.333333         0   
2019-10-01 11:00:00      94.777778         1      0.111111         0   
2019-10-01 11:30:00     141.000000         3      0.230769         1   
2019-10-01 12:00:00     133.454545         0      0.0000

In [6]:
df.shape

(822, 11)

In [4]:
xtrain, ytrain, xtest, ytest = train[:, :4], train[:, 3], test[:, :4], test[:, 3]
print(xtest.shape, ytest.shape)

# lookback = 60
lookback = 1

n_features = 4
train_len = len(xtrain) - lookback
test_len = len(xtest) - lookback
# val_len = len(xval) - lookback

x_train = np.zeros((train_len, lookback, n_features))
y_train = np.zeros((train_len))
for i in range(train_len):
    ytemp = i + lookback
    x_train[i] = xtrain[i:ytemp]
    y_train[i] = ytrain[ytemp]
print("x_train", x_train.shape)
print("y_train", y_train.shape)

x_test = np.zeros((test_len, lookback, n_features))
y_test = np.zeros((test_len))
for i in range(test_len):
    ytemp = i + lookback
    x_test[i] = xtest[i:ytemp]
    y_test[i] = ytest[ytemp]
print("x_test", x_test.shape)
print("y_test", y_test.shape)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=80, verbose=1, mode='min')

(165, 4) (165,)
x_train (656, 1, 4)
y_train (656,)
x_test (164, 1, 4)
y_test (164,)


In [ ]:
import tensorflow
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model


X_train, X_test = x_train, x_test


def build_model(hp):
    model = Sequential()
    # number of hidden layer
    hidden = hp.Int("n_hidden", min_value=0, max_value=3)
    # first LSTM−Layer
    model.add(LSTM(units=hp.Int("n_units1", min_value=17, max_value=500, step=50),
                   activation=hp.Choice("v_activation", values=["relu", "tanh", "sigmoid"], default="relu"),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   return_sequences=True if hidden > 0 else False))

    if hidden > 0:
        for layer in range(hidden):
            model.add(Dropout(hp.Float("v_dropout_hidden" + str(layer + 1), min_value=0.05, max_value=0.95, step=0.05)))
            model.add(LSTM(units=hp.Int("n_units_hidden" + str(layer + 1), min_value=17,
                                        max_value=250,
                                        step=50),
                           activation="relu",
                           return_sequences=True if layer != hidden else False))

    model.add(Dropout(hp.Float("v_dropout", min_value=0.05, max_value=0.95, step=0.05)))
    model.add(Dense(units=1))

    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


tuner = RandomSearch(build_model,
                     objective="val_loss", max_trials=50,
                     seed=1,
                     executions_per_trial=3, directory="",
                     project_name=""
                     )

early_stopping_cb = tensorflow.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tuner.search(X_train, y_train,
             epochs=100,
             # epochs=1500,
             batch_size=32,
             validation_split=0.2, callbacks=[early_stopping_cb], verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]
# save best model for that variable combination
best_model.save("best_model.h5")
# Evaluate the best model with test data
loss = best_model.evaluate(X_test, y_test)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
n_hidden          |0                 |?                 
n_units1          |467               |?                 
v_activation      |relu              |?                 
v_dropout         |0.25              |?                 

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function wi

In [1]:
epochs = 100
batch_size = 32

history = best_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                         validation_split = 0.2,
                        # validation_data=(x_val, y_val),
                         verbose=1,
                        shuffle=False, callbacks=[earlystop])

loss = history.history
plt.plot(loss['loss'])
plt.plot(loss['val_loss'])
plt.savefig("loss_baseline_AAPL.jpg")
plt.show()
y_pred = best_model.predict(x_test)
print("r2_score:", r2_score(y_pred, y_test))

plt.figure(figsize=(20, 10))
plt.plot(y_test, '.-', color='red', label='Real values', alpha=0.5)
plt.plot(y_pred, '.-', color='blue', label='Predicted values', alpha=1)
plt.savefig("baseline_AAPL.jpg")
plt.show()

NameError: name 'best_model' is not defined